In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os


In [5]:
dataset_path = '/content/drive/MyDrive/archive/TrashType_Image_Dataset'


In [6]:
IMG_SIZE = (224, 224)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [7]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # Freeze base layers


In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)  # 6 classes

model = Model(inputs=base_model.input, outputs=predictions)


In [9]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.3169 - loss: 1.6560 - val_accuracy: 0.6541 - val_loss: 1.0344
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.6804 - loss: 0.9131 - val_accuracy: 0.7336 - val_loss: 0.8102
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.7690 - loss: 0.6948 - val_accuracy: 0.7575 - val_loss: 0.7331
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.8210 - loss: 0.5402 - val_accuracy: 0.7614 - val_loss: 0.6937
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.8445 - loss: 0.4738 - val_accuracy: 0.7793 - val_loss: 0.6576
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8724 - loss: 0.4133 - val_accuracy: 0.7793 - val_loss: 0.6461
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8835 - loss: 0.3921

In [16]:
base_model.trainable = True
# But freeze first 100 layers, and train rest
for layer in base_model.layers[:100]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate!
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [13]:
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # You can try 5–15
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.6853 - loss: 0.8220 - val_accuracy: 0.7873 - val_loss: 0.6277
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 190s 3s/step - accuracy: 0.8152 - loss: 0.5348 - val_accuracy: 0.7833 - val_loss: 0.6263
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.8810 - loss: 0.4055 - val_accuracy: 0.7753 - val_loss: 0.6123
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 176s 3s/step - accuracy: 0.8891 - loss: 0.3632 - val_accuracy: 0.7833 - val_loss: 0.6054
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 174s 3s/step - accuracy: 0.9149 - loss: 0.3034 - val_accuracy: 0.7952 - val_loss: 0.5879
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 183s 3s/step - accuracy: 0.9383 - loss: 0.2455 - val_accuracy: 0.7952 - val_loss: 0.5963
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 191s 3s/step - accuracy: 0.9489 - loss: 0.2267 - val_accuracy: 0.8032 - val_loss: 0.5743
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.9574 - loss: 0.1991 - val_accuracy: 0.8111 - v

In [14]:
# Save using new format
model.save('/content/drive/MyDrive/garbage_classifier_model_v2.keras')



In [15]:
from tensorflow import keras

model = keras.models.load_model('/content/drive/MyDrive/garbage_classifier_model_v2.keras', compile=False)

# Then recompile
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
